# Model Engineering

In [1]:
%matplotlib inline

import lmfit
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
import tellurium as te
import requests
import tesbml

## Reading BioModel

In [3]:
def createURL(filenum=FILENUM):
    """
    Creates the URL for the BioModel number.
    :param int filenum:
    :return url:
    """
    return "%s%s" % (URL_BASE, format(filenum, '010d'))

In [4]:
def printReactions(filenum=FILENUM, model=None):
    if model is None:
        model = getModel(filenum)
    numreaction = model.getNumReactions()
    for i in range(numreaction):
        reaction = model.getReaction(i)
        stg = " "

        # print reactants - left hand side of the reaction
        for j in range(reaction.getNumReactants()):
            reactant = reaction.getReactant(j)
            stg = stg + " " + reactant.getSpecies()

        stg = stg + " -> "

        # print products - right hand side of the reaction
        for k in range(reaction.getNumProducts()):
            product = reaction.getProduct(k)
            stg = stg + " " + product.getSpecies()

        stg = stg + ";"

        print(stg)

In [5]:
createURL(64)

'https://www.ebi.ac.uk/biomodels-main/download?mid=BIOMD0000000064'

In [6]:
def getModel(filenum=FILENUM):
    """
    :param str file_num:
    :return libsbml.Model:
    """
    url = createURL(filenum)
    response = requests.get(url).content
    document = tesbml.readSBMLFromString(response.decode("utf-8"))
    return document.getModel()

In [10]:
model=getModel(1)
model.getNumReactions()

17

In [ ]:
model.getReaction(0)

In [9]:
dir(model)

['__class__',
 '__del__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__metaclass__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__swig_destroy__',
 '__swig_getmethods__',
 '__swig_setmethods__',
 '__weakref__',
 '_s',
 'addCVTerm',
 'addCompartment',
 'addCompartmentType',
 'addConstantAttribute',
 'addConstraint',
 'addDefinitionsForDefaultUnits',
 'addEvent',
 'addFunctionDefinition',
 'addInitialAssignment',
 'addModifiers',
 'addParameter',
 'addReaction',
 'addRule',
 'addSpecies',
 'addSpeciesType',
 'addUnitDefinition',
 'all_element_id_list',
 'all_element_meta_id_list',
 'all_elements',
 'all_elements_from_plugins',
 'annotation',
 'annotation_string',
 'appendAnnotation',
 'appendFrom',
 'appendNotes',
 'ar

## Modules

In [18]:
modelstr = """
/* Create instance for each level of phosphorylation */
T2pR => T3p + R; kf*T2pR - kr*T3p*R
T3p => T3 + p;  kf*T3p - kr*T3*p
T3p => T2p + m;  kf*T3p - kr*T2p*m
T2p + R => T2pR;  kf*T2p - kr*T2pR
T3pR => T4p + R; kf*T3pR - kr*T4p*R
T4p => T4 + p;  kf*T4p - kr*T4*p
T4p => T3p + m;  kf*T4p - kr*T3p*m
T3p + R => T3pR;  kf*T3p - kr*T3
T4pR => T5p + R; kf*T4pR - kr*T5p*R
T5p => T5 + p;  kf*T5p - kr*T5*p
T5p => T4p + m;  kf*T5p - kr*T4p*m
T4p + R => T4pR;  kf*T4p - kr*T4


kf = 1
kr = 1
    
"""
r = te.loada(modelstr)
results = r.simulate(0,  20, 50)
results

        time, [T2pR], [T3p], [R], [T3], [p], [T2p], [m], [T3pR], [T4p], [T4]
 [[        0,      0,     0,   0,    0,   0,     0,   0,      0,     0,    0],
  [ 0.408163,      0,     0,   0,    0,   0,     0,   0,      0,     0,    0],
  [ 0.816327,      0,     0,   0,    0,   0,     0,   0,      0,     0,    0],
  [  1.22449,      0,     0,   0,    0,   0,     0,   0,      0,     0,    0],
  [  1.63265,      0,     0,   0,    0,   0,     0,   0,      0,     0,    0],
  [  2.04082,      0,     0,   0,    0,   0,     0,   0,      0,     0,    0],
  [  2.44898,      0,     0,   0,    0,   0,     0,   0,      0,     0,    0],
  [  2.85714,      0,     0,   0,    0,   0,     0,   0,      0,     0,    0],
  [  3.26531,      0,     0,   0,    0,   0,     0,   0,      0,     0,    0],
  [  3.67347,      0,     0,   0,    0,   0,     0,   0,      0,     0,    0],
  [  4.08163,      0,     0,   0,    0,   0,     0,   0,      0,     0,    0],
  [   4.4898,      0,     0,   0,    0,   0,     0,   

In [30]:
modelstr = """
/* Create instance for each level of phosphorylation */
model complex()
    TpR => Tmp + R; kf*TpR - kr*Tmp*R
    Tmp => Tm + p;  kf*Tmp - kr*Tm*p
    Tmp => Tp + m;  kf*Tmp - kr*Tp*m
    Tp + R => TpR;  kf*Tp - kr*TpR
    kf = 1
    kr = 1
end

Me2: complex();
Me3: complex();
Me4: complex();

Me2.Tmp := Me3.Tp
Me2.m := Me3.m
Me2.p := Me3.p
Me3.Tmp := Me4.Tp
Me3.m := Me4.m
Me3.p := Me4.p

Me2.TpR = 10
Me4.m = 10

"""
r = te.loada(modelstr)
results = r.simulate(0,  20, 50)

In [31]:
results

        time,    [Me2_TpR], [Me2_R], [Me2_Tm],     [Me2_Tp], [Me3_TpR], [Me3_R], [Me3_Tm], [Me3_Tp], [Me4_TpR], [Me4_Tmp], [Me4_R], [Me4_Tm], [Me4_p], [Me4_Tp], [Me4_m], [Me2_Tmp], [Me2_p], [Me2_m], [Me3_Tmp], [Me3_p], [Me3_m]
 [[        0,           10,       0,        0,            0,         0,       0,        0,        0,         0,         0,       0,        0,       0,        0,      10,         0,       0,      10,         0,       0,      10],
  [ 0.408163,      4.56932, 5.43068,        0,       0.4898,         0,       0,        0,        0,         0,         0,       0,        0,       0,        0,      10,         0,       0,      10,         0,       0,      10],
  [ 0.816327,      2.11186, 7.88814,        0,     0.231696,         0,       0,        0,        0,         0,         0,       0,        0,       0,        0,      10,         0,       0,      10,         0,       0,      10],
  [  1.22449,     0.976329, 9.02367,        0,     0.107172,         0,       0,      